In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
INTERVAL = 0.005
GRID_LEVELS = 20

def grid_price(level, base_price):
    return int(base_price * math.pow(1 + INTERVAL, level - GRID_LEVELS / 2))

In [3]:
LINE_COUNT = 5
GRID_SPACE = 0.05

def get_grid_lines(current_price):
    stop_loss = current_price - ((LINE_COUNT + 1) * GRID_SPACE)
    take_profit = current_price + ((LINE_COUNT + 1) * GRID_SPACE)

    buy_lines = []
    sell_lines = []

    for i in range(LINE_COUNT):
        buy_lines.append(current_price - (i * GRID_SPACE))
        sell_lines.append(current_price + (i * GRID_SPACE))
         
    return stop_loss, take_profit, buy_lines, sell_lines

In [4]:
def calculate_position_size(acc_size, tick_value=1):
    risk = acc_size * 0.1
    ticks_at_risk = (LINE_COUNT + 1) * GRID_SPACE
    position_size = round(risk / (ticks_at_risk * tick_value), 2)
    return position_size

In [5]:
init_funds = 100000

position_size = calculate_position_size(acc_size=init_funds)
partial_position = position_size / 5

In [6]:
df = pd.read_csv("../data/adausdt.csv")
df.head()

,Opentime,Open,High,Low,Close,Volume,Closetime,Quote asset volume,Number of trades,Taker by base,Taker buy quote,Ignore
0,2021-11-01 08:00:00,1.965,2.019,1.911,1.949,144061135.4,1635811199999,2.824457e+08,443944,71078236.0,1.394258e+08,0
1,2021-11-02 08:00:00,1.949,1.996,1.933,1.971,124263204.0,1635897599999,2.446304e+08,382592,64339347.3,1.267002e+08,0
2,2021-11-03 08:00:00,1.971,2.144,1.950,2.063,335561983.7,1635983999999,6.936646e+08,1237122,167743048.3,3.467647e+08,0
3,2021-11-04 08:00:00,2.063,2.103,1.933,1.983,174549100.4,1636070399999,3.509825e+08,586577,80789835.4,1.624612e+08,0
4,2021-11-05 08:00:00,1.983,2.017,1.959,1.982,108180863.7,1636156799999,2.150281e+08,346939,51873631.3,1.030921e+08,0


In [7]:
stop_loss, take_profit, buy_lines, sell_lines = get_grid_lines(df['Close'][0])

In [8]:
buy_lines = np.asanyarray(buy_lines)
# buy_lines[buy_lines > 0]
np.min(buy_lines)

1.749

In [9]:
%time
buy_count = 0
buy_price = []
sell_price = []

for i in range(0, len(df)):
    if df['Close'][i] > np.min(buy_lines) and df['Close'][i] <= np.max(buy_lines) and buy_count == 0:
        print(f'Buy market price: { df.Close[i] }\t qty: {partial_position}')
        print(f'at time {i}')
        stop_loss, take_profit, buy_lines, sell_lines = get_grid_lines(df['Close'][i])
        print("=======================")
        print("buy lines: ", buy_lines)
        print("sell lines: ", sell_lines)
        print("=======================\n")
        buy_price.append((i, df.Close[i]))
        buy_count += 1

    if df['Close'][i] > np.min(sell_lines) and df['Close'][i] <= np.max(sell_lines) and buy_count > 0:
        print(f'Sell market price: { df.Close[i] }\t qty: {partial_position}')
        print(f'at time {i}')
        stop_loss, take_profit, buy_lines, sell_lines = get_grid_lines(df['Close'][i])
        print("=======================")
        print("buy lines: ", buy_lines)
        print("sell lines: ", sell_lines)
        print("=======================\n")
        sell_price.append((i, df.Close[i]))
        buy_count -= 1

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs
Buy market price: 1.949	 qty: 6666.666
at time 0
buy lines:  [1.949, 1.899, 1.849, 1.799, 1.749]
sell lines:  [1.949, 1.999, 2.049, 2.099, 2.149]

Sell market price: 1.971	 qty: 6666.666
at time 1
buy lines:  [1.971, 1.921, 1.871, 1.8210000000000002, 1.7710000000000001]
sell lines:  [1.971, 2.021, 2.071, 2.121, 2.1710000000000003]

Buy market price: 1.874	 qty: 6666.666
at time 15
buy lines:  [1.874, 1.824, 1.774, 1.7240000000000002, 1.6740000000000002]
sell lines:  [1.874, 1.9240000000000002, 1.9740000000000002, 2.024, 2.0740000000000003]

Sell market price: 1.876	 qty: 6666.666
at time 16
buy lines:  [1.876, 1.8259999999999998, 1.7759999999999998, 1.726, 1.676]
sell lines:  [1.876, 1.926, 1.976, 2.026, 2.076]

Buy market price: 1.786	 qty: 6666.666
at time 17
buy lines:  [1.786, 1.736, 1.686, 1.6360000000000001, 1.586]
sell lines:  [1.786, 1.836, 1.8860000000000001, 1.936, 1.986]

Sell market price: 1.863	 qty: 6666.666


In [10]:
%time
# calculate total profit
sum([y[1] - x[1] for x, y in zip(buy_price, sell_price)])

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


0.10099999999999976